<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Debugging" data-toc-modified-id="Debugging-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Debugging</a></span></li><li><span><a href="#Trainer" data-toc-modified-id="Trainer-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Trainer</a></span></li><li><span><a href="#Losses" data-toc-modified-id="Losses-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Losses</a></span></li><li><span><a href="#VITS-Trainer" data-toc-modified-id="VITS-Trainer-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>VITS Trainer</a></span></li></ul></div>

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [ ]:
# default_exp trainer.gradtts

# Debugging

# Trainer

In [ ]:
# export
import json
import os
from pathlib import Path
from pprint import pprint
import numpy as np

import torch
from torch.cuda.amp import autocast, GradScaler
import torch.distributed as dist
from torch.nn import functional as F
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.optim.lr_scheduler import ExponentialLR
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
import time

from uberduck_ml_dev.models.common import MelSTFT
from uberduck_ml_dev.utils.plot import (
    plot_attention,
    plot_gate_outputs,
    plot_spectrogram,
    plot_tensor,
)
from uberduck_ml_dev.text.util import text_to_sequence, random_utterance
from uberduck_ml_dev.text.symbols import symbols_with_ipa
from uberduck_ml_dev.trainer.base import TTSTrainer

from uberduck_ml_dev.data_loader import (
    TextAudioSpeakerLoader,
    TextMelCollate,
    DistributedBucketSampler,
    TextMelDataset,
)
from uberduck_ml_dev.vendor.tfcompat.hparam import HParams
from uberduck_ml_dev.utils.plot import save_figure_to_numpy, plot_spectrogram
from uberduck_ml_dev.utils.utils import slice_segments, clip_grad_value_
from uberduck_ml_dev.text.symbols import SYMBOL_SETS

# Grad TTS Trainer

In [ ]:
# export
from tqdm import tqdm
from uberduck_ml_dev.text.util import text_to_sequence, random_utterance
from uberduck_ml_dev.models.gradtts import (
    GradTTS,
)
from uberduck_ml_dev.utils.utils import intersperse


class GradTTSTrainer(TTSTrainer):
    REQUIRED_HPARAMS = [
        "training_audiopaths_and_text",
        "test_audiopaths_and_text",
    ]

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        for param in self.REQUIRED_HPARAMS:
            if not hasattr(self, param):
                raise Exception(f"GradTTSTrainer missing a required param: {param}")
        self.sampling_rate = self.hparams.sampling_rate
        self.checkpoint_path = self.hparams.log_dir

    def sample_inference(self, model, timesteps=10, spk=None):
        with torch.no_grad():
            sequence = text_to_sequence(
                random_utterance(),
                self.text_cleaners,
                1.0,
                symbol_set=self.hparams.symbol_set,
            )
            if self.hparams.intersperse_text:
                sequence = intersperse(
                    sequence, (len(SYMBOL_SETS[self.hparams.symbol_set]))
                )
            x = torch.LongTensor(sequence).cuda()[None]
            x_lengths = torch.LongTensor([x.shape[-1]]).cuda()
            y_enc, y_dec, attn = model(
                x,
                x_lengths,
                n_timesteps=50,
                temperature=1.5,
                stoc=False,
                spk=spk,
                length_scale=0.91,
            )
            audio = self.sample(
                y_dec,
                algorithm="hifigan",
                hifigan_config="/home/w_uberduck_ai/Speech-Backbones/Grad-TTS/checkpts/hifigan-config.json",
                hifigan_checkpoint="/home/w_uberduck_ai/Speech-Backbones/Grad-TTS/checkpts/gen_02640000_studio",
                max_wav_value=32768,
                cudnn_enabled=self.hparams.cudnn_enabled,
            )
            return audio

    def train(self, checkpoint=None):
        if self.distributed_run:
            self.init_distributed()

        train_dataset = TextMelDataset(
            self.hparams.training_audiopaths_and_text,
            self.hparams.text_cleaners,
            1.0,
            self.hparams.n_feats,
            self.hparams.sampling_rate,
            self.hparams.mel_fmin,
            self.hparams.mel_fmax,
            self.hparams.filter_length,
            self.hparams.hop_length,
            (self.hparams.filter_length - self.hparams.hop_length) // 2,
            self.hparams.win_length,
            intersperse_text=self.hparams.intersperse_text,
            intersperse_token=(len(SYMBOL_SETS[self.hparams.symbol_set])),
            symbol_set=self.hparams.symbol_set
            #             debug=self.debug,
            #             debug_dataset_size=self.debug_dataset_size,
        )
        collate_fn = TextMelCollate()

        loader = DataLoader(
            dataset=train_dataset,
            batch_size=self.hparams.batch_size,
            collate_fn=collate_fn,
            drop_last=True,
            num_workers=0,
            shuffle=False,
        )

        test_dataset = TextMelDataset(
            self.hparams.test_audiopaths_and_text,
            self.hparams.text_cleaners,
            1.0,
            self.hparams.n_feats,
            self.hparams.sampling_rate,
            self.hparams.mel_fmin,
            self.hparams.mel_fmax,
            self.hparams.filter_length,
            self.hparams.hop_length,
            (self.hparams.filter_length - self.hparams.hop_length) // 2,
            self.hparams.win_length,
            intersperse_text=self.hparams.intersperse_text,
            intersperse_token=(len(SYMBOL_SETS[self.hparams.symbol_set])),
            symbol_set=self.hparams.symbol_set,
            #             debug=self.debug,
            #             debug_dataset_size=self.debug_dataset_size,
        )

        model = GradTTS(self.hparams)

        if self.hparams.checkpoint:
            model.load_state_dict(torch.load(self.hparams.checkpoint))
        model = model.cuda()

        print(
            "Number of encoder + duration predictor parameters: %.2fm"
            % (model.encoder.nparams / 1e6)
        )
        print("Number of decoder parameters: %.2fm" % (model.decoder.nparams / 1e6))
        print("Total parameters: %.2fm" % (model.nparams / 1e6))

        print("Initializing optimizer...")
        optimizer = torch.optim.Adam(
            params=model.parameters(), lr=self.hparams.learning_rate
        )
        test_batch = test_dataset.sample_test_batch(size=self.hparams.test_size)
        for i, item in enumerate(test_batch):
            text, mel, spk = item
            self.log(
                f"image_{i}/ground_truth",
                0,
                image=plot_tensor(mel.squeeze()),
            )

        iteration = 0
        last_time = time.time()
        for epoch in range(1, self.hparams.n_epochs + 1):
            model.train()
            dur_losses = []
            prior_losses = []
            diff_losses = []
            for batch_idx, batch in enumerate(loader):
                model.zero_grad()
                x, x_lengths, y, _, y_lengths, speaker_ids = batch

                dur_loss, prior_loss, diff_loss = model.compute_loss(
                    x, x_lengths, y, y_lengths, out_size=self.hparams.out_size
                )
                loss = sum([dur_loss, prior_loss, diff_loss])
                loss.backward()

                enc_grad_norm = torch.nn.utils.clip_grad_norm_(
                    model.encoder.parameters(), max_norm=1
                )
                dec_grad_norm = torch.nn.utils.clip_grad_norm_(
                    model.decoder.parameters(), max_norm=1
                )
                optimizer.step()

                self.log("training/duration_loss", iteration, dur_loss.item())
                self.log("training/prior_loss", iteration, prior_loss.item())
                self.log("training/diffusion_loss", iteration, diff_loss.item())
                self.log("training/encoder_grad_norm", iteration, enc_grad_norm)
                self.log("training/decoder_grad_norm", iteration, dec_grad_norm)

                dur_losses.append(dur_loss.item())
                prior_losses.append(prior_loss.item())
                diff_losses.append(diff_loss.item())

                iteration += 1

            log_msg = f"Epoch {epoch}, iter: {iteration}: dur_loss: {np.mean(dur_losses):.4f} | prior_loss: {np.mean(prior_losses):.4f} | diff_loss: {np.mean(diff_losses):.4f} | time: {time.time()-last_time:.2f}s"
            last_time = time.time()
            with open(f"{self.hparams.log_dir}/train.log", "a") as f:
                f.write(log_msg + "\n")
                print(log_msg)

            if epoch % self.log_interval == 0:
                model.eval()
                with torch.no_grad():
                    for i, item in enumerate(test_batch):
                        x, _y, _speaker_id = item
                        x = x.to(torch.long).unsqueeze(0)
                        x_lengths = torch.LongTensor([x.shape[-1]])
                        y_enc, y_dec, attn = model(x, x_lengths, n_timesteps=50)
                        self.log(
                            f"image_{i}/generated_enc",
                            iteration,
                            image=plot_tensor(y_enc.squeeze().cpu()),
                        )
                        self.log(
                            f"image_{i}/generated_dec",
                            iteration,
                            image=plot_tensor(y_dec.squeeze().cpu()),
                        )
                        self.log(
                            f"image_{i}/alignment",
                            iteration,
                            image=plot_tensor(attn.squeeze().cpu()),
                        )
                        self.log(
                            f"audio/inference_{i}",
                            iteration,
                            audio=self.sample_inference(model),
                        )

            if epoch % self.save_every == 0:
                torch.save(
                    model.state_dict(), f=f"{self.hparams.log_dir}/grad_{epoch}.pt"
                )

In [ ]:
DEFAULTS = HParams(
    training_audiopaths_and_text="/home/w_uberduck_ai/uberduck-rappers/lachow_sanity.txt",
    test_audiopaths_and_text="/home/w_uberduck_ai/uberduck-rappers/lachow_sanity.txt",
    cudnn_enabled=True,
    log_dir="output",
    symbol_set="gradtts",
    intersperse_text=True,
    n_spks=1,
    spk_emb_dim=64,
    sampling_rate=22050,
    hop_length=256,
    win_length=1024,
    n_enc_channels=192,
    filter_channels=768,
    filter_channels_dp=256,
    n_enc_layers=6,
    enc_kernel=3,
    enc_dropout=0.1,
    n_heads=2,
    window_size=4,
    dec_dim=64,
    beta_min=0.05,
    beta_max=20.0,
    pe_scale=1000,
    test_size=2,
    n_epochs=10000,
    batch_size=1,
    learning_rate=1e-4,
    seed=37,
    out_size=2 * 22050 // 256,
    filter_length=1024,
    rank=0,
    distributed_run=False,
    oversample_weights=None,
    text_cleaners=["english_cleaners"],
    max_wav_value=32768.0,
    n_feats=80,
    mel_fmax=8000,
    mel_fmin=0.0,
    checkpoint=None,
    log_interval=100,
    save_every=1000,
)
trainer = GradTTSTrainer(DEFAULTS, rank=0)

TTSTrainer start 2012310.788795182
Initializing trainer with hparams:
{'batch_size': 1,
 'beta_max': 20.0,
 'beta_min': 0.05,
 'checkpoint': None,
 'cudnn_enabled': True,
 'dec_dim': 64,
 'distributed_run': False,
 'enc_dropout': 0.1,
 'enc_kernel': 3,
 'filter_channels': 768,
 'filter_channels_dp': 256,
 'filter_length': 1024,
 'hop_length': 256,
 'intersperse_text': True,
 'learning_rate': 0.0001,
 'log_dir': 'output',
 'log_interval': 100,
 'max_wav_value': 32768.0,
 'mel_fmax': 8000,
 'mel_fmin': 0.0,
 'n_enc_channels': 192,
 'n_enc_layers': 6,
 'n_epochs': 10000,
 'n_feats': 80,
 'n_heads': 2,
 'n_spks': 1,
 'out_size': 172,
 'oversample_weights': None,
 'pe_scale': 1000,
 'rank': 0,
 'sampling_rate': 22050,
 'save_every': 1000,
 'seed': 37,
 'spk_emb_dim': 64,
 'symbol_set': 'gradtts',
 'test_audiopaths_and_text': '/home/w_uberduck_ai/uberduck-rappers/lachow_sanity.txt',
 'test_size': 2,
 'text_cleaners': ['english_cleaners'],
 'training_audiopaths_and_text': '/home/w_uberduck_ai

In [ ]:
trainer.train()

Number of encoder + duration predictor parameters: 7.20m
Number of decoder parameters: 7.63m
Total parameters: 14.84m
Initializing optimizer...
Epoch 1, iter: 5: dur_loss: 0.6656 | prior_loss: 15.6823 | diff_loss: 0.9930 | time: 0.63s
Epoch 2, iter: 10: dur_loss: 0.6328 | prior_loss: 14.8335 | diff_loss: 0.7924 | time: 0.53s
Epoch 3, iter: 15: dur_loss: 0.6274 | prior_loss: 13.5213 | diff_loss: 0.7031 | time: 0.51s
Epoch 4, iter: 20: dur_loss: 0.4993 | prior_loss: 14.7112 | diff_loss: 0.5350 | time: 0.48s
Epoch 5, iter: 25: dur_loss: 0.5745 | prior_loss: 12.6364 | diff_loss: 0.4708 | time: 0.55s
Epoch 6, iter: 30: dur_loss: 0.5692 | prior_loss: 12.2626 | diff_loss: 0.3381 | time: 0.49s
Epoch 7, iter: 35: dur_loss: 0.5588 | prior_loss: 12.1942 | diff_loss: 0.2707 | time: 0.54s
Epoch 8, iter: 40: dur_loss: 0.5193 | prior_loss: 12.2359 | diff_loss: 0.2822 | time: 0.53s
Epoch 9, iter: 45: dur_loss: 0.4628 | prior_loss: 11.9533 | diff_loss: 0.2984 | time: 0.49s
Epoch 10, iter: 50: dur_loss:

KeyboardInterrupt: 